In [1]:
import sys
import os
sys.path.append('/data/GitHub/lca-tools/')

import lcatools

In [2]:
from lcatools.entities import NoReferenceFound, NoAllocation

In [3]:
sys.path.append('..')
import lcamatrix

In [4]:
CATALOG = '/data/GitHub/lca-catalog/catalogs/'

EI32_A = 'ecoinvent_3.2_apos_spold.json.gz'
#USLCI = 'uslci_ecospold.json.gz'

In [5]:
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [6]:
EA = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.zip', prefix='datasets')

Found Extension: zip


In [7]:
EA.load_all()

Loading /data/LCI/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.zip
 Loaded 100 processes (t=1.03 s)
 Loaded 200 processes (t=2.01 s)
 Loaded 300 processes (t=2.67 s)
 Loaded 400 processes (t=3.25 s)
 Loaded 500 processes (t=3.63 s)
activity ID 0a3fc02d-87cb-4200-88ac-ce5fc0ad5f42: no comment
 Loaded 600 processes (t=4.51 s)
 Loaded 700 processes (t=5.12 s)
 Loaded 800 processes (t=5.69 s)
 Loaded 900 processes (t=6.66 s)
 Loaded 1000 processes (t=7.23 s)
 Loaded 1100 processes (t=7.73 s)
 Loaded 1200 processes (t=8.51 s)
 Loaded 1300 processes (t=9.55 s)
 Loaded 1400 processes (t=10.23 s)
 Loaded 1500 processes (t=10.78 s)
 Loaded 1600 processes (t=11.85 s)
 Loaded 1700 processes (t=12.51 s)
 Loaded 1800 processes (t=12.99 s)
 Loaded 1900 processes (t=13.88 s)
 Loaded 2000 processes (t=15.24 s)
 Loaded 2100 processes (t=16.00 s)
 Loaded 2200 processes (t=16.99 s)
 Loaded 2300 processes (t=18.15 s)
 Loaded 2400 processes (t=18.65 s)
 Loaded 2500 processes (t=19.32 s)
 Loaded 2600 p

In [8]:
EA.write_to_file(os.path.join(CATALOG, EI32_A), exchanges=True, values=True, characterizations=True, gzip=True)

In [7]:
%time EA_j = lcatools.archive_from_json(os.path.join(CATALOG, EI32_A))

Loading JSON data from /data/GitHub/lca-catalog/catalogs/ecoinvent_3.2_apos_spold.json.gz:
Found Extension: zip


AmbiguousReferenceError: Allocation key is not a reference exchange

In [8]:
len(EA.processes())

11420

In [9]:
ps = EA.search(entity_type='process', Name='yogurt')

In [10]:
str(ps[0]) == 'yogurt production, from cow milk [CA-QC]'

True

In [11]:
p = ps[0]

In [12]:
p.get_uuid() == '580dc5e1-8866-4caa-848e-456476fde542'

True

In [13]:
for x in p.reference_entity:
    print(str(x))

yogurt production, from cow milk [CA-QC] has Output: cream, from cow milk [By-product classification: allocatable product] [kg]
yogurt production, from cow milk [CA-QC] has Output: cheese, from cow milk, fresh, unripened [By-product classification: allocatable product] [kg]
yogurt production, from cow milk [CA-QC] has Output: yogurt, from cow milk [By-product classification: allocatable product] [kg]


In [14]:
yog = p.find_reference('yogurt')

In [15]:
yog

In [47]:
import importlib

In [50]:
importlib.reload(lcamatrix.foreground)

<module 'lcamatrix.foreground' from '../lcamatrix/foreground.py'>

In [51]:
from lcamatrix.foreground import BackgroundManager

In [52]:
B = BackgroundManager(EA)

In [31]:
import cProfile

In [62]:
37.782 - 19.522 - 7.111 - 3.904 - 1.211 - 1.16 - 2.217 - .453 
# _traverse - exchanges - terminate - add_cutoff - add_interior - _set_lowlink - _check_product_flow - _create_product_flow
# leaves remainder:

2.203999999999997

In [53]:
cProfile.run('B.add_ref_product(yog.flow, p)')

         83863151 function calls (81764818 primitive calls) in 39.341 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:402(parent)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000   39.341   39.341 <string>:1(<module>)
   603418    0.208    0.000    0.449    0.000 <string>:12(__new__)
        4    0.000    0.000    0.001    0.000 _methods.py:25(_amax)
        2    0.000    0.000    0.000    0.000 _methods.py:28(_amin)
        3    0.000    0.000    0.000    0.000 base.py:1081(isspmatrix)
       16    0.000    0.000    0.000    0.000 base.py:181(nnz)
        1    0.000    0.000    0.075    0.075 base.py:236(asformat)
        1    0.000    0.000    0.010    0.010 base.py:253(processes)
        1    0.002    0.002    0.010    0.010 base.py:254(<listcomp>)
        4    0.000    

In [54]:
%time bg = [b.index for b in B.tstack.background_flows()]

CPU times: user 3.33 ms, sys: 3.33 ms, total: 6.67 ms
Wall time: 6.26 ms


In [55]:
%time bg_dict = dict((v, k) for k, v in enumerate(bg))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.07 ms


In [56]:
%time B._construct_b_matrix(bg_dict)

CPU times: user 690 ms, sys: 20 ms, total: 710 ms
Wall time: 708 ms


In [58]:
B._cutoff

[MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e21de80>, value=0.00391255795812271),
 MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e037208>, value=5.25921928345733e-09),
 MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e0cf7b8>, value=1.33893853637212e-07)]

In [60]:
.865 + .678 + 37.782

39.324999999999996

In [34]:
pd = EA['55ee5166-8fd9-4955-937c-69b0a66f30ee']

In [37]:
refs = list(pd.reference_entity)

In [49]:
pd.is_allocated(refs[0])

True

In [56]:
len(F._cutoff) + len(F._interior) == 791758

True

In [59]:
[len(t) for t in F.tstack._sccs.values()]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10097,
 1]

In [68]:
pf = list(F._product_flows.keys())

In [74]:
F.tstack.scc_id(pf[3336])

1

In [79]:
['%s: %s' % (pf[i]._process['Name'], pf[i]._flow['Name']) for i in (3337, 3338)]

['market for electricity, low voltage: electricity, low voltage',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted: electricity, low voltage']

In [80]:
from collections import defaultdict

In [81]:
d = defaultdict(set)

In [84]:
def make_scc_dict(matrix):
    # exchs is a list of MatrixEntries (parent, term, exch)
    # corresponds to column, row, value
    # we want a list of nonzero columns for each row
    d = defaultdict(set)
    for i in matrix._interior:
        d[matrix.tstack.scc_id(i.term)].add(matrix.tstack.scc_id(i.parent))
    return d

In [99]:
import json

In [107]:
ind = None
ml = 0
for k, t in F.tstack._sccs.items():
    if len(t) > ml:
        ind = k
        ml = len(t)

In [118]:
len(F._interior)

353648

In [122]:
co = F._cutoff[33700]

In [135]:
len(F._cutoff)

438109

In [134]:
F._cutoff.remove(co)

In [132]:
co.exchange.direction

'Output'

In [110]:
len(None)

TypeError: object of type 'NoneType' has no len()

In [100]:
json.dumps(F._interior[0])

TypeError: Object of type 'ProductFlow' is not JSON serializable

In [85]:
%time d = make_scc_dict(F)

CPU times: user 510 ms, sys: 0 ns, total: 510 ms
Wall time: 510 ms


In [90]:
[str(p._process) for p in F.tstack._sccs[815]]

['treatment of lignite ash, opencast refill [DE]']

In [93]:
ds = list(F.tstack._sccs[815])[0]

In [96]:
ds._process.inventory()

treatment of lignite ash, opencast refill [DE]
 0 Output: {*} [-1 [kg]] lignite ash [By-product classification: Waste]
 1 Output:     [ ---  [kg]] Tungsten (CAS 007440-33-7) [ground-]
 2 Output:     [ ---  [kg]] Chloride (CAS 016887-00-6) [ground-, long-term]
 3 Output:     [ ---  [kg]] Antimony (CAS 007440-36-0) [ground-, long-term]
 4 Output:     [ ---  [kg]] Nickel, ion (CAS 014701-22-5) [ground-, long-term]
 5 Output:     [ ---  [kg]] Arsenic, ion (CAS 017428-41-0) [ground-]
 6 Output:     [ ---  [kg]] Strontium (CAS 007440-24-6) [ground-, long-term]
 7 Output:     [ ---  [kg]] Selenium (CAS 007782-49-2) [ground-, long-term]
 8 Output:     [ ---  [kg]] Sodium, ion (CAS 017341-25-2) [ground-]
 9 Output:     [ ---  [kg]] Calcium, ion (CAS 014127-61-8) [ground-, long-term]
10 Output:     [ ---  [kg]] Chromium VI (CAS 018540-29-9) [ground-]
11 Output:     [ ---  [kg]] Strontium (CAS 007440-24-6) [ground-]
12 Output:     [ ---  [kg]] Fluoride (CAS 016984-48-8) [ground-]
13 Output:     [

In [87]:
d

defaultdict(set,
            {1: {0, 1},
             28: {1},
             29: {28},
             177: {1},
             178: {177},
             197: {1},
             242: {1},
             243: {1},
             277: {1},
             278: {277},
             379: {1},
             380: {379},
             523: {1},
             524: {523},
             525: {523},
             526: {1},
             527: {526},
             528: {526},
             547: {1},
             548: {1},
             676: {1},
             677: {1},
             678: {1},
             812: {1},
             813: {1},
             814: {1},
             815: {1},
             816: {1},
             817: {1},
             818: {1},
             819: {1},
             820: {1},
             821: {1},
             822: {1},
             823: {1},
             912: {1},
             913: {912},
             914: {912},
             1034: {1},
             1035: {1},
             1036: {1},
             1260: 

In [46]:
%time [str(r._process) for r in F._product_flows if r not in d]

CPU times: user 6.67 ms, sys: 0 ns, total: 6.67 ms
Wall time: 7.87 ms


['yogurt production, from cow milk [CA-QC]']

In [30]:
791758 / 2800

282.7707142857143

In [31]:
38 / 282

0.1347517730496454

In [22]:
[len([f for f in F.scc(s)]) for s in F.sccs]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10097,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [19]:
len(F.sccs)

187

In [23]:
pb = EA['f3622e6c-b364-4108-9574-925359b64f51']

In [20]:
xs = [x for x in EA.exchanges(EA['3a9b33f8-d1b1-4bea-9df7-7db6846c6df6'])]

In [35]:
len(xs)

8

In [38]:
x_cand = [x for x in xs if x.termination == pb.uuid]
x_excl = [x for x in xs if x.termination != pb.uuid]

In [46]:
x_cand[0].flow.show()


Flow Entity (ref 3a9b33f8-d1b1-4bea-9df7-7db6846c6df6)
       Name: samarium europium gadolinium concentrate, 94% rare earth oxide
    Comment: EcoSpold02 Flow
  CasNumber: 
Compartment: ['By-product classification: allocatable product']


In [43]:
[(str(x.termination), EA[x.termination]) for x in x_excl]

[('bb2eff39-6693-49bc-b735-3762193f0039',
  <lcatools.entities.LcProcess at 0x7f1749dffbe0>),
 ('8d44da1b-8c02-4640-a965-55ca57bc8bfe',
  <lcatools.entities.LcProcess at 0x7f174b229438>),
 ('None', None),
 ('None', None),
 ('None', None),
 ('4296051c-2908-4a6c-ac19-f1dd2bc7c9b9',
  <lcatools.entities.LcProcess at 0x7f174dc0e048>)]

In [22]:
[str(x) for x in xs]

[' Input:     [ ---  [kg]] samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product] (#)',
 ' Input:     [ ---  [kg]] samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product] (#)',
 'Output: {*} [1 [kg]] samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product]',
 'rare earth oxides production from bastnäsite concentrate [RoW] has Output: samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product] [kg]',
 'Output:     [ ---  [kg]] samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product] (#)',
 'Output:     [ ---  [kg]] samarium europium gadolinium concentrate, 94% rare earth oxide [By-product classification: allocatable product] (#)',
 'rare earth oxides production from bastnäsite concentrate [CN] has Output: samarium eu

In [61]:
[x.__dict__ for p in EA.search('8d44da1b') for x in p.exchange(EA['3a9b33f8-d1b1-4bea-9df7-7db6846c6df6'])]

[{'_direction': 'Output',
  '_flow': <lcatools.entities.LcFlow at 0x7f174dc0e390>,
  '_process': <lcatools.entities.LcProcess at 0x7f174b229438>,
  '_termination': None,
  'unit': <lcatools.entities.LcUnit at 0x7f1750f64588>},
 {'_direction': 'Output',
  '_flow': <lcatools.entities.LcFlow at 0x7f174dc0e390>,
  '_process': <lcatools.entities.LcProcess at 0x7f174b229438>,
  '_termination': 'f3622e6c-b364-4108-9574-925359b64f51',
  '_value': None,
  '_value_dict': {<lcatools.exchanges.Exchange at 0x7f174b229b00>: 1.0},
  'unit': <lcatools.entities.LcUnit at 0x7f1750f64588>}]

In [57]:
print('[%s] %s %s -> %s %s (%s)' % ('parent process', 'dirn-wrt-parent', 'term', 'resolved term', 'ref?', 'match?'))
for x in xs:
    t = x.termination
    p = EA[t]
    if p is None:
        r = ''
        is_ref = False
    else:
        #try:
        r = len([z for z in p.reference_entity if z.flow == x.flow])
        #    r = '+'
        #except NoReferenceFound:
        #    r = 'xxxx'
        is_ref = x in x.process.reference_entity
    print('[%s] %1.1s %s -> %s %s (%s)' % (x.process.uuid, x.direction, t, p, is_ref, r))

[parent process] dirn-wrt-parent term -> resolved term ref? (match?)
[4296051c-2908-4a6c-ac19-f1dd2bc7c9b9] I bb2eff39-6693-49bc-b735-3762193f0039 -> rare earth oxides production from bastnäsite concentrate [CN] False (1)
[4296051c-2908-4a6c-ac19-f1dd2bc7c9b9] I 8d44da1b-8c02-4640-a965-55ca57bc8bfe -> rare earth oxides production from bastnäsite concentrate [RoW] False (1)
[4296051c-2908-4a6c-ac19-f1dd2bc7c9b9] O None -> None False ()
[8d44da1b-8c02-4640-a965-55ca57bc8bfe] O None -> None False ()
[8d44da1b-8c02-4640-a965-55ca57bc8bfe] O f3622e6c-b364-4108-9574-925359b64f51 -> samarium europium gadolinium concentrate to generic market for mischmetal [GLO] False (0)
[bb2eff39-6693-49bc-b735-3762193f0039] O f3622e6c-b364-4108-9574-925359b64f51 -> samarium europium gadolinium concentrate to generic market for mischmetal [GLO] False (0)
[bb2eff39-6693-49bc-b735-3762193f0039] O None -> None False ()
[f3622e6c-b364-4108-9574-925359b64f51] I 4296051c-2908-4a6c-ac19-f1dd2bc7c9b9 -> market for s

In [53]:
[x.flow.show() for x in xs[4].process.references()]

Flow Entity (ref cfbc8873-47d4-4411-a18e-578ef35ab871)
       Name: lanthanum oxide
    Comment: EcoSpold02 Flow
  CasNumber: 001312-81-8
Compartment: ['By-product classification: allocatable product']
Flow Entity (ref df12f8f5-a3e3-4506-9d86-b33205d21a52)
       Name: cerium concentrate, 60% cerium oxide
    Comment: EcoSpold02 Flow
  CasNumber: 
Compartment: ['By-product classification: allocatable product']
Flow Entity (ref 361e78e0-0094-48d2-b9a2-2c70921e5c1e)
       Name: neodymium oxide
    Comment: EcoSpold02 Flow
  CasNumber: 001313-97-9
Compartment: ['By-product classification: allocatable product']
Flow Entity (ref 3a9b33f8-d1b1-4bea-9df7-7db6846c6df6)
       Name: samarium europium gadolinium concentrate, 94% rare earth oxide
    Comment: EcoSpold02 Flow
  CasNumber: 
Compartment: ['By-product classification: allocatable product']
Flow Entity (ref e1126956-dc57-4837-849f-57fb25c8effa)
       Name: praseodymium oxide
    Comment: EcoSpold02 Flow
  CasNumber: 012037-29-5
Compa

[None, None, None, None, None]

In [24]:
pb.show()

Process Entity (ref f3622e6c-b364-4108-9574-925359b64f51)
           Name: samarium europium gadolinium concentrate to generic market for mischmetal
        Comment: This dataset provides an outlet for surplus samarium europium gadolinium concentrate from co-production with other rare earth oxides. This dataset implies an assumption that a surplus will add to the content of these elements in the general mischmetal (mixed rare earths). This route should be replaced by a dataset representing the marginal application of the samarium europium gadolinium concentrate, as soon as this becomes available.
Classifications: ['EcoSpold01Categories: chemicals/inorganics', 'ISIC rev.4 ecoinvent: 2011:Manufacture of basic chemicals']
  TemporalScope: {'end': '2015-12-31', 'begin': '2011-01-01'}
   SpatialScope: GLO


In [31]:
RX = next(x for x in pb.reference_entity)

In [34]:
RX.flow.show()

Flow Entity (ref 474e7001-a158-4379-8793-e02f0c14bbfe)
       Name: mischmetal
    Comment: EcoSpold02 Flow
  CasNumber: 
Compartment: ['By-product classification: allocatable product']


In [32]:
rf = EA['3a9b33f8-d1b1-4bea-9df7-7db6846c6df6']

In [26]:
for r in pb.reference(EA['3a9b33f8-d1b1-4bea-9df7-7db6846c6df6']):
    print(str(r))
    if r.termination is not None:
        print('TERM: \n' % r.termination)

NoReferenceFound: 

In [33]:
import os
import sys
import json
import uuid 

from itertools import groupby

In [48]:
def group_by(iterable, keyfunc):
    groups = []
    uniquekeys = []
    data = sorted(iterable, key=keyfunc)
    for k, g in groupby(data, keyfunc):
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
        
    return groups, uniquekeys

In [3]:
import re
import gzip

from eight import USING_PYTHON2


def from_json(fname):
    """
    Routine to reconstruct a catalog from a json archive.
    :param fname: json file, optionally gzipped
    :return: a subclass of ArchiveInterface
    """
    print('Loading JSON data from %s:' % fname)
    if bool(re.search('\.gz$', fname)):
        if USING_PYTHON2:
            with gzip.open(fname, 'r') as fp:
                j = json.load(fp)
        else:
            with gzip.open(fname, 'rt') as fp:
                j = json.load(fp)
    else:
        with open(fname, 'r') as fp:
            j = json.load(fp)
    return j


In [17]:
us = from_json(os.path.join(CATALOG, USLCI))

Loading JSON data from /data/GitHub/lca-catalog/catalogs/uslci_ecospold.json.gz:


In [23]:
ei = from_json(os.path.join(CATALOG, EI32_A))

Loading JSON data from /data/GitHub/lca-catalog/catalogs/ecoinvent_3.2_apos_spold.json.gz:


In [18]:
us.keys()

dict_keys(['quantities', '@context', 'dataSourceType', 'processes', 'catalogNames', 'flows', 'dataSourceReference', 'nsUuid'])

## utility functions

these would be methods of an lca-catalog class, should I ever get around to making one that is independent of the `lca-tools` infrastructure, should there be a need.

In [28]:
def make_flows(catalog):
    flows = dict()
    for f in catalog['flows']:
        flows[f['entityId']] = f
    return flows


## detect interior flows

The strategy here is to make two sets of UUIDs- of inputs and outputs- and then find their intersection

In [20]:
def interior_flows(catalog):
    ins = set()
    outs = set()
    for p in catalog['processes']:
        for x in p['exchanges']:
            if x['direction'] == 'Input':
                ins.add(x['flow'])
            elif x['direction'] == 'Output':
                outs.add(x['flow'])
            else:
                raise ValueError('mysterious direction designation "%s"' % x['direction'])
    print('Found %d inputs, %d outputs' % (len(ins), len(outs)))
    interior = ins.intersection(outs)
    print('Found %d interior flows' % len(interior))
    return interior
                    

In [13]:
ei['processes'][0]['exchanges'][0]['direction']

'Output'

In [24]:
%time in_ei = interior_flows(ei)

Found 2971 inputs, 4355 outputs
Found 2688 interior flows
CPU times: user 210 ms, sys: 0 ns, total: 210 ms
Wall time: 204 ms


In [27]:
us['flows'][430]

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'unspecified'],
 'Name': 'Chromium III',
 'characterizations': [{'entityType': 'characterization',
   'isReference': True,
   'quantity': 'bbe8f43d-c141-3e21-b8c2-3ba1ce556ad0',
   'value': 1.0}],
 'entityId': '1c4485a2-5ac9-3c06-aca9-25aabf72e45c',
 'entityType': 'flow',
 'externalId': 9483,
 'origin': '/home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip'}

In [26]:
us.keys()

dict_keys(['quantities', '@context', 'dataSourceType', 'processes', 'catalogNames', 'flows', 'dataSourceReference', 'nsUuid'])

In [29]:
us_flows = make_flows(us)

In [30]:
len(us_flows)

4176

In [49]:
gs, ks = group_by(us_flows.values(), lambda x: x['Compartment'][0])

In [52]:
ks

['Air Transportation',
 'Biofuels Manufacturing',
 'CUTOFF Flows',
 'Chemical Manufacturing',
 'Crop Production',
 'Elec. Equip., Appliance, and Comp. Manufacturing',
 'Elementary Flows',
 'Fabricated Metal Product Manufacturing',
 'Forestry and Logging',
 'Mining (except Oil and Gas)',
 'Nonmetallic Mineral Product Manufacturing',
 'Oil and Gas Extraction',
 'Paper Manufacturing',
 'Petroleum and Coal Products Manufacturing',
 'Plastics and Rubber Products Manufacturing',
 'Primary Metal Manufacturing',
 'Rail Transportation',
 'Transit and Ground Passenger Transportation',
 'Transportation Equipment Manufacturing',
 'Truck Transportation',
 'Utilities',
 'Waste Management and Remediation Services',
 'Water Transportation',
 'Wood Product Manufacturing',
 'air',
 'final-waste-flow',
 'non-material',
 'others',
 'resource',
 'soil',
 'water']

In [60]:
for i, g in enumerate(gs):
    print(' %2d %30.30s: %6d' % (i, ks[i], len(g)))

  0             Air Transportation:      1
  1         Biofuels Manufacturing:      7
  2                   CUTOFF Flows:    523
  3         Chemical Manufacturing:    128
  4                Crop Production:     55
  5 Elec. Equip., Appliance, and C:     10
  6               Elementary Flows:     19
  7 Fabricated Metal Product Manuf:     12
  8           Forestry and Logging:    111
  9    Mining (except Oil and Gas):      7
 10 Nonmetallic Mineral Product Ma:      5
 11         Oil and Gas Extraction:      3
 12            Paper Manufacturing:     49
 13 Petroleum and Coal Products Ma:     11
 14 Plastics and Rubber Products M:     22
 15    Primary Metal Manufacturing:     50
 16            Rail Transportation:      1
 17 Transit and Ground Passenger T:     46
 18 Transportation Equipment Manuf:      5
 19           Truck Transportation:    102
 20                      Utilities:    127
 21 Waste Management and Remediati:     22
 22           Water Transportation:      6
 23     Woo

In [62]:
for f in gs[13]:
    print(f['Name'])

Petroleum refining, at refinery
Diesel, at refinery
Residual fuel oil, at refinery
Petroleum refining coproduct, unspecified, at refinery
Kerosene, at refinery
Liquefied petroleum gas, at refinery
Gasoline, at refinery
Petroleum refining coproduct, at refinery
Refinery gas, at refinery
Bitumen, at refinery
Petroleum coke, at refinery


In [167]:
sorted([us_flows[f]['Name'].lower() for f in in_us if us_flows[f]['Name'].lower().find('combusted') >= 0])

['anthracite coal, combusted in industrial boiler',
 'bituminous coal, combusted in industrial boiler',
 'bituminous coal, combusted in industrial boiler, at pulp and paper mill (excl.)',
 'diesel, combusted in industrial boiler',
 'diesel, combusted in industrial boiler, at pulp and paper mill (excl.)',
 'diesel, combusted in industrial equipment',
 'gasoline, combusted in equipment',
 'gasoline, combusted in equipment, at pulp and paper mill (excl.)',
 'hog fuel, pur., combusted in industrial boiler, at pulp and paper mill (excl.)',
 'hog fuel, self-gen., combusted in ind. boiler, at pulp and paper mill (excl.)',
 'lignite coal, combusted in industrial boiler',
 'liquefied petroleum gas, combusted in industrial boiler',
 'lpg, combusted in industrial boiler, at pulp and paper mill (excl.)',
 'natural gas, combusted in industrial boiler',
 'natural gas, combusted in industrial boiler, at hydrocracker, for butadiene',
 'natural gas, combusted in industrial boiler, at hydrocracker, for 

In [171]:
us['nsUuid']

'a9d158e0-d48c-4427-8c55-42719e9e11cc'

In [176]:
[p for p in us['processes'] if p['entityId'] == str(uuid.uuid3(uuid.UUID(us['nsUuid']), 'Petroleum refining, at refinery'))]



[{'Classifications': ['Petroleum and Coal Products Manufacturing',
   'Petroleum Refineries'],
  'Comment': 'This module expresses data on the basis 1 kilogram of general refinery product as well as data allocated to specific refinery products. The data are allocated to specific refinery products based on the percent by mass of each product in the refinery output. Refinery products include gasoline, distillate/diesel oil, LPG, residual oil, kerosene/jet fuel, and other (still gas, petroleum, coke, asphalt, and petrochemical feedstocks).  Complete inventory data and metadata are available in full in the final report and appendices, Cradle-to-Gate Life Cycle Inventory of Nine Plastic Resins and Four Polyurethane Precursors. This report has been extensively reviewed within Franklin Associates and has undergone partial critical review by ACC Plastics Division members and is available at: www.americanchemistry.com. Quantities may vary slightly between the reference to main source and and th

In [85]:
next(f for f in us_flows.values())['Name'].lower().find('diesel')

-1

In [68]:
g.find('ds')

2

In [89]:
from collections import Counter

In [103]:
elem = {'air', 'water', 'soil', 'resource'}

output_counter = Counter()

for p in us['processes']:
    for x in p['exchanges']:
        if x['direction'] == 'Output':
            if x['flow'] in in_us:
            #if us_flows[x['flow']]['Compartment'][0] not in elem:
                output_counter[x['flow']] += 1
            

In [ ]:
us_flows

In [182]:
for k in output_counter.most_common(30):
    print('%2d: (%s) %s' % (k[1], us_flows[k[0]]['externalId'], us_flows[k[0]]['Name']))

86: (5853) Chromium
61: (6619) Chloride
54: (6621) Sodium, ion
50: (5875) Carbon dioxide, biogenic
19: (6615) Chromium, ion
16: (6601) Thallium
 8: (6605) Hydrogen cyanide (prussic acid)
 8: (6603) Chloride (unspecified)
 8: (7907) Inorganic salts and acids, unspecified
 8: (9688) Water, cooling
 7: (6617) Methane, bromotrifluoro-, Halon 1301
 7: (15462) Cooling water, non-contact
 3: (13968) Water
 2: (775) Diesel, at refinery
 2: (6403) Oil, crude, 42 MJ per kg, in ground
 2: (16700) Kerosene, at refinery
 2: (7295) Quartz sand (silica sand; silicon dioxide)
 2: (7293) Sodium chloride, in ground
 2: (16696) Residual fuel oil, at refinery
 2: (7352) Lignite, 11 MJ per kg, in ground
 2: (13406) Petroleum coke, at refinery
 2: (16694) Liquefied petroleum gas, at refinery
 2: (6903) Oxygen
 2: (6401) Gas, natural, 46.8 MJ per kg, in ground
 2: (7350) Coal, hard, 30.7 MJ per kg, in ground
 2: (23293) Harvesting, fresh fruit bunch, at farm
 2: (869) Gasoline, at refinery
 2: (5339) Wood fu

In [111]:
us_flows[output_counter.most_common(4)[3][0]]

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'unspecified'],
 'Name': 'Carbon dioxide, biogenic',
 'characterizations': [{'entityType': 'characterization',
   'isReference': True,
   'quantity': 'bbe8f43d-c141-3e21-b8c2-3ba1ce556ad0',
   'value': 1.0}],
 'entityId': '80d22a4f-d79f-3597-9e4e-23c25505d5ad',
 'entityType': 'flow',
 'externalId': 5875,
 'origin': '/home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip'}

In [115]:
co2_biogenic =output_counter.most_common(4)[3][0]
chromium = output_counter.most_common(1)[0][0]

In [118]:
us_flows['11183286-23c2-3521-8b55-4907c5754aaf']

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['resource', 'in ground'],
 'Name': 'Chromium, 25.5 in chromite, 11.6% in crude ore, in ground',
 'characterizations': [{'entityType': 'characterization',
   'isReference': True,
   'quantity': 'bbe8f43d-c141-3e21-b8c2-3ba1ce556ad0',
   'value': 1.0}],
 'entityId': '11183286-23c2-3521-8b55-4907c5754aaf',
 'entityType': 'flow',
 'externalId': 6570,
 'origin': '/home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip'}

In [119]:
[p['Classifications'] for p in us['processes'] if chromium in [x['flow'] for x in p['exchanges'] if x['direction'] =='Input']]

[['Primary Metal Manufacturing', 'Primary Metal Manufacturing'],
 ['Primary Metal Manufacturing', 'Primary Metal Manufacturing']]

In [125]:
def set_to_index(flow_set):
    """
    Creates a dict of flow UUID to enumeration.  there's probably an easier [pythonic] way to do this.
    """
    index = dict()
    count = 0
    for f in flow_set:
        index[f] = count
        count +=1
    return index

In [122]:
import scipy as sp

In [124]:
from scipy.sparse import coo_matrix

In [143]:
_process_iter = (p for p in us['processes'])

In [126]:
p = next(p for p in us['processes'])

In [130]:
def reference_exchanges(p):
    """
    returns an iterable of exchange dicts that match x['isReference'] is True 
    """
    for x in p['exchanges']:
        if 'isReference' in x.keys():
            if x['isReference'] is True:
                yield x

In [146]:
[p for p in us['processes'] if len([x for x in reference_exchanges(p)]) < 1]

[{'Classifications': ['Paper Manufacturing',
   'Pulp, Paper, and Paperboard Mills'],
  'Comment': 'Uncoated Freesheet, average North American production, at mill.  By-products were allocated 0% of the impacts to enable the user to choose and apply their own allocation method more easily. All flows are production-weighted means. Data were developed for a full LCA that was critically reviewed.  Reviewers were: • Martha Stevenson, Chairperson – Private Consultant to the public interest with ten years of experience in sustainability, environmental management, packaging and materials recovery; led the development of the Design Guidelines for Sustainable Packaging and the COMPASS software used to compare the environmental impacts of package designs; • Pascal Lesage – Life cycle assessment consultant, Researcher at École Polytechnique de Montréal [at the Inter-University Research Centre for the Life Cycle of Products, Processes and Services (CIRAIG) in Montreal]; and • Dale Phenicie, Environ

In [123]:
def create_technology_matrix(catalog):
    interior = interior_flows(catalog)
    index = set_to_index(interior)  # this is a dict that maps interior flow to index
    

AttributeError: module 'scipy' has no attribute 'sparse'

In [149]:
len(in_ei)

2688

In [150]:
ei_flows = make_flows(ei)

In [151]:
[ei_flows[x]['Name'] for x in in_ei if ei_flows[x]['Name'].lower().find('electricity') >= 0 ]

['operation, computer, laptop, video mode, label-certified electricity',
 'electricity, for reuse in municipal waste incineration only',
 'energy use and operation emissions, electric bicycle, label-certified electricity',
 'operation, internet access equipment, label-certified electricity',
 'mini CHP plant, common components for heat+electricity',
 'electricity, low voltage',
 'heat and power co-generation unit, 6400kW thermal, components for electricity only',
 'transmission network, electricity, high voltage',
 'heat and power co-generation unit, 1MW electrical, components for electricity only',
 'operation, computer, laptop, 68% active work, label-certified electricity',
 'electricity, high voltage, for internal use in coal mining',
 'heat and power co-generation unit, 200kW electrical, diesel SCR, common components for heat+electricity',
 'transport, passenger, electric bicycle, label-certified electricity',
 'heat and power co-generation unit, 6400kW thermal, common components f

In [164]:
['%-8.8s %s' % (p['SpatialScope'], p['Name']) for p in ei['processes'] if re.search('^market group', p['Name'], flags=re.I)]

['RME      market group for electricity, high voltage',
 'RER      market group for heavy fuel oil',
 'CA       market group for electricity, high voltage',
 'GLO      market group for natural gas, high pressure',
 'RAS      market group for electricity, high voltage',
 'RAS      market group for electricity, medium voltage',
 'GLO      market group for diesel',
 'GLO      market group for transport, freight train',
 'Canada w market group for electricity, high voltage',
 'US       market group for electricity, medium voltage',
 'RER      market group for heat, district or industrial, other than natural gas',
 'RER      market group for tap water',
 'Europe w market group for electricity, high voltage',
 'UCTE     market group for electricity, low voltage',
 'Canada w market group for electricity, low voltage',
 'ENTSO-E  market group for electricity, high voltage',
 'GLO      market group for tap water',
 'RNA      market group for electricity, medium voltage',
 'US       market group